In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoTokenizer, AutoModel
import torch
from collections import Counter

# BERT 토크나이저와 모델 초기화
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")
if torch.cuda.is_available():
    model.cuda()

In [2]:
# 장르 목록을 문자열로 변환하는 함수
def get_genres_text(data):
    genres_text = []
    for genres_list in data:
        genres_text.append(' '.join(genres_list))
    return genres_text


In [3]:
# 데이터를 로드하고 전처리하는 함수
def load_and_preprocess_data(file_path):
    # 로드
    data = pd.read_csv(file_path)

    # 데이터 전처리
    data['title_overview'] = data['title'] + ' ' + data['overview']
    data['title_overview'] = data['title_overview'].str.lower()
    data.drop(['Unnamed: 0', 'id', 'status', 'original_language', 'original_title', 'spoken_languages', 'overview'],
                axis=1, inplace=True)
    data = data[data.runtime < 300].reset_index(drop=True)
    data['year'] = data['release_date'].str[:4]
    data['month'] = data['release_date'].str[5:7]
    data.drop(['release_date'], axis=1, inplace=True)
    years_to_remove = [2024, 2025, 2026, 2027, 2099]
    data = data[~data['year'].astype(int).isin(years_to_remove)].reset_index(drop=True)
    data['genres'] = data['genres'].str.split(',').apply(lambda x: [genre.strip().lower() for genre in x])
    data['production_countries'] = data['production_countries'].str.split(',').apply(lambda x: [country.strip().lower() for country in x])
    top_countries = [country for country, _ in Counter(data['production_countries'].sum()).most_common(9)]
    data['production_countries'] = data['production_countries'].apply(lambda x: [country if country in top_countries else 'Other' for country in x])
    data = data[data['revenue'] > 1000000]
    data['genres_text'] = get_genres_text(data['genres'])
    return data

In [4]:
# 장르 기반으로 상위 영화를 필터링하는 함수
def filter_top_movies_by_genre(top_movies_df, df, top_n=20):
    # 코사인 유사도
    top_movies_vectors = np.array(list(top_movies_df['genres_text_vector']))
    all_movies_vectors = np.array(list(df['genres_text_vector']))

    cos_similarities = cosine_similarity(top_movies_vectors, all_movies_vectors)

    # 필터링
    top_indices = [np.argsort(cos_similarities[i])[::-1][:top_n] for i in range(len(top_movies_df))]

    # 데이터 프레임 생성
    top_movies_by_genre = pd.DataFrame()
    for indices in top_indices:
        top_movies_by_genre = pd.concat([top_movies_by_genre, df.iloc[indices]])

    return top_movies_by_genre.drop_duplicates(subset=['title', 'vote_average', 'revenue', 'popularity', 'vote_count', 'title_overview', 'genres_text']).head(top_n)


In [6]:
# Function to compute BERT embeddings
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    if torch.cuda.is_available():
        inputs = {k: v.to('cuda') for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)

    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

In [7]:
def recommend_movies(movie_title, df, top_n=5):
    # 선택한 영화의 벡터를 가져옵니다.
    selected_movie_vector = df[df['title'] == movie_title]['title_overview_vector'].values[0]

    # 모든 영화의 벡터와 선택한 영화의 벡터 간의 코사인 유사도를 계산합니다.
    cos_similarities = cosine_similarity([selected_movie_vector], list(df['title_overview_vector']))[0]

    # 코사인 유사도가 가장 높은 상위 50개 영화의 인덱스를 가져옵니다.
    top_50_indices = np.argsort(cos_similarities)[::-1][1:51]

    # 상위 50개 영화의 데이터 프레임을 생성합니다.
    top_50_movies = df.iloc[top_50_indices]

    # 상위 50개 영화 중 장르가 유사한 상위 20개 영화를 필터링합니다.
    top_20_movies = filter_top_movies_by_genre(top_50_movies, df)

    # 상위 20개 영화 중 수익, 투표 수, 평균 투표 점수가 가장 높은 상위 5개 영화를 선택합니다.
    top_5_movies = top_20_movies.sort_values(by=['revenue', 'vote_count', 'vote_average'], ascending=False).head(top_n)

    # 상위 5개 영화의 제목을 반환합니다.
    return top_5_movies['title'].values

# 데이터를 로드하고 전처리합니다.
df = load_and_preprocess_data("data.csv")

# 영화 개요와 장르 텍스트에 대해 BERT 임베딩을 계산합니다.
df['title_overview_vector'] = df['title_overview'].apply(lambda x: get_bert_embeddings(x)[0])
df['genres_text_vector'] = df['genres_text'].apply(lambda x: get_bert_embeddings(x)[0])

# 파이프라인을 사용하여 영화를 추천합니다.
recommended_movies = recommend_movies("The Eyes", df)
print(recommended_movies)

['Insidious: Chapter 3' 'The Menu' 'The Last Exorcism'
 'Escape Room: Tournament of Champions' 'Ready or Not']


In [8]:
# 영화 제목을 변수에 저장합니다.
movie_name = "The Eyes"

# 영화 제목을 사용하여 영화 추천을 실행합니다.
# 추천 영화는 코사인 유사도를 사용하여 선택되며, 장르가 유사한 영화가 우선적으로 선택됩니다.
# 또한, 수익, 투표 수, 평균 투표 점수가 높은 영화가 우선적으로 선택됩니다.
recommended_movies = recommend_movies(movie_name, df)

# 추천된 영화 목록을 출력합니다.
# 각 영화 제목 앞에 '+' 기호를 추가하여 강조합니다.
print(f"Movies that related to {movie_name} : ")
for i in recommended_movies:
    print(f"+{i}")


Movies that related to The Eyes : 
+Insidious: Chapter 3
+The Menu
+The Last Exorcism
+Escape Room: Tournament of Champions
+Ready or Not


In [9]:
movie_name = "The Avengers"
print(f"Movies that related to {movie_name} : ")
recommended_movies = recommend_movies(movie_name, df)
for i in recommended_movies:   
    print(f"+{i}")

Movies that related to The Avengers : 
+Avengers: Age of Ultron
+Black Panther
+Iron Man 3
+Man of Steel
+Justice League


In [10]:
movie_name = "The Amazing Spider-Man"
print(f"Movies that related to {movie_name} : ")
recommended_movies = recommend_movies(movie_name, df)
for i in recommended_movies:   
    print(f"+{i}")

Movies that related to The Amazing Spider-Man : 
+Aquaman
+The Hobbit: The Battle of the Five Armies
+Batman v Superman: Dawn of Justice
+Wonder Woman
+The Amazing Spider-Man


In [11]:
movie_name = "Mommy"
print(f"Movies that related to {movie_name} : ")
recommended_movies = recommend_movies(movie_name, df)
for i in recommended_movies:   
    print(f"+{i}")

Movies that related to Mommy : 
+Bridge of Spies
+Money Monster
+Margin Call
+Neerja
+The Words


In [12]:
movie_name = "Seoul Station"
print(f"Movies that related to {movie_name} : ")
recommended_movies = recommend_movies(movie_name, df)
for i in recommended_movies:   
    print(f"+{i}")

Movies that related to Seoul Station : 
+Mission: Impossible - Ghost Protocol
+The Mummy
+The First Purge
+The Purge: Election Year
+Red Riding Hood
